In [1]:
import os
import openai
from dotenv import load_dotenv

from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import AzureSearch
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

import warnings
warnings.filterwarnings("ignore")

## Load Environmental Variables

In [2]:
# Load environment variables
load_dotenv()

# Configure OpenAI API
openai.api_type = "azure"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_version = os.getenv('OPENAI_API_VERSION')

## Connect to Azure Search Index where Documents are Stored

In [3]:
# Initialize gpt-35-turbo and our embedding model
llm = AzureChatOpenAI(deployment_name="gpt-35-turbo")
embeddings = OpenAIEmbeddings(deployment_id="text-embedding-ada-002", chunk_size=1)

# Connect to Azure Cognitive Search
acs = AzureSearch(azure_search_endpoint=os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT'),
                 azure_search_key=os.getenv('AZURE_SEARCH_API_KEY'),
                 index_name=os.getenv('AZURE_SEARCH_INDEX_NAME'),
                 embedding_function=embeddings.embed_query)

## Define a QA Chain Using LangChain

In [4]:
# Ref: https://clemenssiebler.com/posts/azure-search-vector-search-openai-langchain/
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=acs.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)

In [5]:
def ask_question(question, chat_history = []):
    result = qa({"question": question, "chat_history": chat_history})

    return result["answer"] #, result["source_documents"]

question = "Can Adaptive Highbeam Assist take into account weather conditions?"
chat_history = [] #(query, result["answer"])
answer = ask_question(question, chat_history)
answer

'No, Adaptive Highbeam Assist cannot take into account weather conditions. Detection may be restricted in poor visibility, such as fog, heavy rain or snow.'

In [6]:
def bot_model(test_set):
    return [ask_question(question_i) for question_i in test_set]

## AdaTest

In [7]:
import adatestt

In [8]:
generator = adatestt.generators.OpenAI(
        model="gpt-35-turbo",
        api_key=os.getenv('OPENAI_API_KEY'),
        api_type="azure",
        api_base=os.getenv('OPENAI_API_BASE'),
        api_version=os.getenv('OPENAI_API_VERSION')
)

In [9]:
tests = adatestt.TestTree("adatest_mva.csv")

In [10]:
adatestt.serve(tests.adapt(bot_model, generator=generator, auto_save=True, max_suggestions=5), host='127.0.0.1', port=8080)

Server started at http://127.0.0.1:8080


Cannot decrypt cookie value, create a new fresh session
